# Decision Tree

## Notation

- $p_1$ - The fraction of correctly classified labels on a node in the decision tree.
- $p_0 = 1 - p_1$ - The fraction of incorrectly classified labels on a node in the decision tree.
- $p_1^{\text{left}}$, $p_1^{\text{right}}$ - Fraction of positive (correct) labels on the left or right subtree respectively.
- $w^{\text{left}}$, $w^{\text{right}}$ - Fraction of all examples that went to the left or right subtree respectively.
- $H(p_1)$ - The *entropy* on a node in the decision tree. Entropy measures the homogeneity of the node.

## Formulas

### Entropy

$H(p_1) = -p_1 \log_2(p_1) - p_0 \log_2(p_0)$

### Information Gain

Information gain refers to the reduction in entropy from making a split.

$H(p_1^{\text{root}}) - (w^{\text{left}} H(p_1^{\text{left}}) + w^{\text{right}} H(p_1^{\text{right}}))$

In [1]:
import numpy as np

## Decision Tree Implementation

In [2]:
class DecisionTree:
    def __init__(self, max_depth) -> None:
        self.tree = []
        self.max_depth = max_depth

    def _build_tree(self, X, y, indices, current_depth):
        if current_depth == self.max_depth:
            return
        
        best_feature = self._get_best_split(X, y, indices)
        left_indices, right_indices = self._split_node(X, indices, best_feature)
        self.tree.append((left_indices, right_indices, best_feature))
        self._build_tree(X, y, left_indices, self.max_depth, current_depth + 1)
        self._build_tree(X, y, right_indices, self.max_depth, current_depth + 1)

    def _get_best_split(self, X, y, indices):
        n_features = X.shape[1]
        best_feature = -1

        max_gain = 0
        for feature in range(n_features):
            information_gain = self._information_gain(X, y, indices)
            if information_gain > max_gain:
                max_gain = information_gain
                best_feature = feature
        
        return best_feature

    def _information_gain(self, X, y, indices, feature):
        left_indices, right_indices = self._split_node(X, indices, feature)
        X_node, y_node = X[indices], y[indices]
        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[right_indices], y[right_indices]

        entropy_root = self._entropy(y_node)
        entropy_left = self._entropy(y_left)
        entropy_right = self._entropy(y_right)
        w_left = len(X_left) / len(X_node)
        w_right = len(X_right) / len(X_node)

        return entropy_root - (w_left * entropy_left + w_right * entropy_right)

    def _split_node(
        X: np.ndarray, indices: list, feature: int
    ) -> tuple[list[int], list[int]]:
        """
        Splits the data at the given node into left and right branches.

        Args:
            X (np.ndarray): Data matrix of shape (n_samples, n_features).
            indices (list): List containing active indices.
            feature (int): Index to split feature on.

        Returns:
            tuple[list[int], list[int]]: _description_
        """
        left_indices = []
        right_indices = []

        for i in indices:
            if X[i, feature] == 1:
                left_indices.append(i)
            else:
                right_indices.append(i)

        return left_indices, right_indices

    def _entropy(self, y: np.ndarray) -> float:
        """
        Computes the entropy for a given node.

        Args:
            y (np.ndarray): NumPy array indicating whether each example at
                            a given node is positive (1) or negative (0).

        Returns:
            float: Entropy value.
        """
        # If all examples are either positive or negative, entropy is 0.
        if sum(y) == 0 or sum(y) == len(y):
            return 0

        p1 = sum(y) / len(y)
        p0 = 1 - p1
        return -p1 * np.log2(p1) - p0 * np.log2(p0)